In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

#### Getting the data

In [2]:
ticker = input("Enter the symbol you want to analyze: ")

In [3]:
ticker = yf.Ticker(ticker)

In [4]:
Balance_sheet = ticker.balance_sheet.transpose()
Income_statement = ticker.income_stmt.transpose()

#### Calculating P/E ratio
    - using diluted EPS

In [5]:
latest_price = float(ticker.history(period='1d')['Close'].values[-1])

In [6]:
ratios = latest_price / Income_statement['Diluted EPS']

In [7]:
ratios = pd.DataFrame(ratios)
ratios.rename({'Diluted EPS': 'P/E ratio'}, axis =1, inplace=True )

#### Calculating price/Book ratio

In [10]:
market_capitalization = Balance_sheet['Ordinary Shares Number'] * latest_price
# calculating net book value 
current_liabilities = Balance_sheet['Current Liabilities'] + Balance_sheet['Other Current Liabilities'] + Balance_sheet['Current Deferred Liabilities']
non_current_liabilities = Balance_sheet['Non Current Deferred Liabilities'] + Balance_sheet['Other Non Current Liabilities'] + Balance_sheet['Non Current Deferred Liabilities']
Balance_sheet['Total Liabilities'] = current_liabilities + non_current_liabilities
Net_book_value = Balance_sheet['Total Assets'] - Balance_sheet['Total Liabilities'] - Balance_sheet['Total Equity Gross Minority Interest']


In [11]:
ratios['Price/Book'] = market_capitalization / Net_book_value

In [12]:
ratios

,P/E ratio,Price/Book
2023-06-30,45.15599,287.817969
2022-06-30,45.296372,142.564515
2021-06-30,54.299377,106.404752
2020-06-30,75.88715,70.009725
